In [2]:
import pandas as pd
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.ar_model import AutoReg
from pymongo import MongoClient
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from glob import glob

### The goal is to figure out if time series model such as ARIMA or AutoReg will be better predictive models of Air Quality (PM2.5) in Lagos/Nigeria than a regular Linear or Ridge regression

Data were downloaded from https://open.africa/dataset/sensorsafrica-airquality-archive-lagos. Turns out, all the data look a bit untidy and sperated with a ";" rather than a "," among other issues.

In [19]:
data_files = glob("./data/tmp*.csv")

In [20]:
data_files[0]
pd.read_csv(data_files[0]).head(2)

,sensor_id;sensor_type;location;lat;lon;timestamp;value_type;value
0,4852;DHT22;3627;6.515;3.400;2024-02-01T12:02:5...
1,4852;DHT22;3627;6.515;3.400;2024-02-01T12:02:5...


In [81]:
temp_df = pd.read_csv(data_files[4])
columns = temp_df.columns.str.split(";")
temp_df = temp_df["sensor_id;sensor_type;location;lat;lon;timestamp;value_type;value"].str.split(";", expand=True)
temp_df.columns = list(columns)[0]
temp_df.tail(20)

,sensor_id,sensor_type,location,lat,lon,timestamp,value_type,value
90112,4854,DHT22,3628,6.559,3.386,2024-03-18T18:49:11.233004+00:00,humidity,78.10
90113,4854,DHT22,3628,6.559,3.386,2024-03-18T18:49:11.233004+00:00,temperature,32.80
90114,4853,pms5003,3628,6.559,3.386,2024-03-18T18:50:05.071658+00:00,P2,21.40
90115,4853,pms5003,3628,6.559,3.386,2024-03-18T18:50:05.071658+00:00,P1,23.60
90116,4853,pms5003,3628,6.559,3.386,2024-03-18T18:50:05.071658+00:00,P0,15.40
90117,4854,DHT22,3628,6.559,3.386,2024-03-18T18:50:18.211645+00:00,humidity,78.60
90118,4854,DHT22,3628,6.559,3.386,2024-03-18T18:50:18.211645+00:00,temperature,32.80
90119,4853,pms5003,3628,6.559,3.386,2024-03-18T18:51:09.080683+00:00,P2,23.33
90120,4853,pms5003,3628,6.559,3.386,2024-03-18T18:51:09.080683+00:00,P1,26.00
90121,4853,pms5003,3628,6.559,3.386,2024-03-18T18:51:09.080683+00:00,P0,17.67


In [87]:
temp_df[temp_df["location"] == "3628"].min()

sensor_id                                  4853
sensor_type                               DHT22
location                                   3628
lat                                       6.559
lon                                       3.386
timestamp      2024-03-01T00:00:39.896476+00:00
value_type                                   P0
value                                     10.00
dtype: object

In [90]:
temp_df[temp_df["location"] == "3630"].min()

sensor_id                                  4857
sensor_type                               DHT22
location                                   3630
lat                                       6.540
lon                                       3.297
timestamp      2024-03-01T11:01:10.225034+00:00
value_type                                   P0
value                                      0.00
dtype: object

In [82]:
temp_df["location"].value_counts()

location
3628    73685
3630    16447
Name: count, dtype: int64

In [6]:
import pytz
temp_df["timestamp"] = pd.to_datetime(temp_df["timestamp"])
temp_df = temp_df.set_index("timestamp")
temp_df.index.tz_convert("Africa/Lagos")[:5]

DatetimeIndex(['2024-02-01 13:02:52.744148+01:00',
               '2024-02-01 13:02:52.744148+01:00',
               '2024-02-01 13:03:47.839650+01:00',
               '2024-02-01 13:03:47.839650+01:00',
               '2024-02-01 13:04:46.909811+01:00'],
              dtype='datetime64[ns, Africa/Lagos]', name='timestamp', freq=None)

In [7]:
temp_df.value_type.value_counts()

value_type
P2             4270
P1             4270
P0             4270
humidity       4144
temperature    4143
Name: count, dtype: int64

In [8]:
## Starting off with the Ridge Regression Model
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import mean_absolute_error

In [9]:
# Dealing with data wrangling
def wrangle(data_file):
    df = pd.read_csv(data_file)
    columns = df.columns.str.split(";")
    df = df["sensor_id;sensor_type;location;lat;lon;timestamp;value_type;value"].str.split(";", expand=True)
    df.columns = list(columns)[0]

    # Working with only PM2.5 value_type
    df_p2 = df["value_type"] == "P2"
    df = df[df_p2]

    # Remove irrelevant columns
    df.drop(columns=['sensor_id','sensor_type', 'location', 'value_type'], inplace=True)

    # Convert timeezone to Lagos's
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df = df.set_index("timestamp")
    df.index.tz_convert("Africa/Lagos")
    
    return df

In [22]:
data_files

['./data\\tmp29zv95aa.csv',
 './data\\tmpdhpbd64k.csv',
 './data\\tmpf7lx946x.csv',
 './data\\tmpoh50r5me.csv',
 './data\\tmpq5nzwa0h.csv']

In [29]:
wrangle(data_files[4]).head()

,lat,lon,value
timestamp,,,
2024-03-01 00:00:39.896476+00:00,6.559,3.386,29.67
2024-03-01 00:01:47.310570+00:00,6.559,3.386,27.00
2024-03-01 00:02:53.783885+00:00,6.559,3.386,31.00
2024-03-01 00:04:01.176132+00:00,6.559,3.386,32.00
2024-03-01 00:05:09.205034+00:00,6.559,3.386,26.50


In [28]:
for data in data_files:
    df = wrangle(data)
    print(df.shape)

(4270, 3)
(9393, 3)
(4304, 3)
(250, 3)
(18272, 3)


In [25]:
merge_order = [3, 1, 2, 0, 4]
data_frames = []
for i in merge_order:
    data_frames.append(wrangle(data_files[i]))

In [26]:
type(data_frames)

list

In [63]:
merged_df = pd.concat(data_frames)

In [64]:
merged_df.shape

(36489, 3)

In [65]:
merged_df.head()

,lat,lon,value
timestamp,,,
2023-11-01 16:44:52.764459+00:00,6.428,3.435,19.00
2023-11-01 17:52:08.924982+00:00,6.540,3.297,3.50
2023-11-01 17:53:13.805911+00:00,6.540,3.297,3.00
2023-11-01 18:16:42.515782+00:00,6.559,3.386,40.33
2023-11-03 07:38:39.664224+00:00,6.540,3.297,47.00


In [66]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 36489 entries, 2023-11-01 16:44:52.764459+00:00 to 2024-03-18 18:53:20.437257+00:00
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   lat     36489 non-null  object
 1   lon     36489 non-null  object
 2   value   36489 non-null  object
dtypes: object(3)
memory usage: 1.1+ MB


##### Converting Colums to float type. (Should have done so in the Wrangle function)

In [67]:
merged_df[["lat", "lon", "value"]] = merged_df[["lat", "lon", "value"]].astype("float")

In [68]:
merged_df.head()

,lat,lon,value
timestamp,,,
2023-11-01 16:44:52.764459+00:00,6.428,3.435,19.00
2023-11-01 17:52:08.924982+00:00,6.540,3.297,3.50
2023-11-01 17:53:13.805911+00:00,6.540,3.297,3.00
2023-11-01 18:16:42.515782+00:00,6.559,3.386,40.33
2023-11-03 07:38:39.664224+00:00,6.540,3.297,47.00


In [69]:
Deciding to resample data airquality data to 1 hour mark rather than within randon minutes.

SyntaxError: invalid syntax (1766138652.py, line 1)

In [70]:
merged_df = merged_df.resample("1H").mean()
merged_df.head()

,lat,lon,value
timestamp,,,
2023-11-01 16:00:00+00:00,6.428,3.435,19.00
2023-11-01 17:00:00+00:00,6.540,3.297,3.25
2023-11-01 18:00:00+00:00,6.559,3.386,40.33
2023-11-01 19:00:00+00:00,NaN,NaN,NaN
2023-11-01 20:00:00+00:00,NaN,NaN,NaN


In [71]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3315 entries, 2023-11-01 16:00:00+00:00 to 2024-03-18 18:00:00+00:00
Freq: H
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   lat     1271 non-null   float64
 1   lon     1271 non-null   float64
 2   value   1271 non-null   float64
dtypes: float64(3)
memory usage: 103.6 KB


In [72]:
merged_df.isnull().sum()/len(merged_df)

lat      0.616591
lon      0.616591
value    0.616591
dtype: float64

#### It appears some periods have missing data after resampling. Since these missing values are over 50% of the data frames total number of rows

In [73]:
new_df = merged_df.dropna()
new_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1271 entries, 2023-11-01 16:00:00+00:00 to 2024-03-18 18:00:00+00:00
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   lat     1271 non-null   float64
 1   lon     1271 non-null   float64
 2   value   1271 non-null   float64
dtypes: float64(3)
memory usage: 39.7 KB


In [ ]:
plt.plot(new_df[""])